In [1]:
from src.ModelTransfuser import ModelTransfuser

import numpy as np
import torch
from scipy.stats import norm
import os

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def load_data(name):
    # --- Load in training data ---
    path_training = os.getcwd() + f'/data/model_comp_data(AGB,SN2,SN1a)/{name}_train.npz'
    training_data = np.load(path_training, mmap_mode='r')

    train_theta = training_data['params']
    train_x = training_data['abundances']

    # ---  Load in the validation data ---
    path_test = os.getcwd() + f'/data/model_comp_data(AGB,SN2,SN1a)/{name}_val.npz'
    val_data = np.load(path_test, mmap_mode='r')

    val_theta = val_data['params']
    val_x = val_data['abundances']

    # --- Clean the data ---
    # Chempy sometimes returns zeros or infinite values, which need to removed
    def clean_data(x, y):
        # Remove all zeros from the training data
        index = np.where((y == 0).all(axis=1))[0]
        x = np.delete(x, index, axis=0)
        y = np.delete(y, index, axis=0)

        # Remove all infinite values from the training data
        index = np.where(np.isfinite(y).all(axis=1))[0]
        x = x[index]
        y = y[index]

        # Remove H from Elements
        y = np.delete(y, 2, 1)

        return x, y

    train_theta, train_x = clean_data(train_theta, train_x)
    val_theta, val_x     = clean_data(val_theta, val_x)

    # convert to torch tensors
    train_theta = torch.tensor(train_theta, dtype=torch.float32)
    train_x = torch.tensor(train_x, dtype=torch.float32)
    val_theta = torch.tensor(val_theta, dtype=torch.float32)
    val_x = torch.tensor(val_x, dtype=torch.float32)

    # --- add noise ---
    pc_ab = 5 # percentage error in abundance

    train_x_err = torch.ones_like(train_x)*float(pc_ab)/100.
    train_x = norm.rvs(loc=train_x,scale=train_x_err)
    train_x = torch.tensor(train_x).float()

    val_x_err = torch.ones_like(val_x)*float(pc_ab)/100.
    val_x = norm.rvs(loc=val_x,scale=val_x_err)
    val_x = torch.tensor(val_x).float()

    # --- Concatenate the data ---
    # train_data = torch.cat((train_theta, train_x), 1)
    # val_data = torch.cat((val_theta, val_x), 1)

    return train_theta, train_x, val_theta, val_x


In [3]:
names = [name.replace("_train.npz","") for name in os.listdir("data/model_comp_data(AGB,SN2,SN1a)/") if "train" in name]

In [4]:
MTf = ModelTransfuser()

In [5]:
for name in names:
    # --- Load in the data ---
    train_theta, train_x, val_theta, val_x = load_data(name)

    MTf.add_data(name, train_theta, train_x, val_theta, val_x)

Data added to model Karakas_net_yield_CL18_net_Seitenzahl
Data added to model Karakas_net_yield_Frischknecht16_net_Seitenzahl
Data added to model Karakas_net_yield_Nomoto2013_Seitenzahl
Data added to model Karakas_net_yield_Nomoto2013_net_Seitenzahl
Data added to model Karakas_net_yield_NuGrid_net_Seitenzahl
Data added to model Karakas_net_yield_Portinari_net_Seitenzahl
Data added to model Karakas_net_yield_TNG_net_Seitenzahl
Data added to model Karakas_net_yield_West17_net_Seitenzahl
Data added to model Karakas_net_yield_chieffi04_Seitenzahl
Data added to model Karakas_net_yield_chieffi04_net_Seitenzahl
Data added to model Nugrid_CL18_net_Seitenzahl
Data added to model Nugrid_Frischknecht16_net_Seitenzahl
Data added to model Nugrid_Nomoto2013_Seitenzahl
Data added to model Nugrid_Nomoto2013_net_Seitenzahl
Data added to model Nugrid_NuGrid_net_Seitenzahl
Data added to model Nugrid_Portinari_net_Seitenzahl
Data added to model Nugrid_TNG_net_Seitenzahl
Data added to model Nugrid_West17_n

In [6]:
MTf.init_models(sde_type="vesde", sigma=2.5, hidden_size=36, depth=5, num_heads=1, mlp_ratio=3)

Models initialized: ['Karakas_net_yield_CL18_net_Seitenzahl', 'Karakas_net_yield_Frischknecht16_net_Seitenzahl', 'Karakas_net_yield_Nomoto2013_Seitenzahl', 'Karakas_net_yield_Nomoto2013_net_Seitenzahl', 'Karakas_net_yield_NuGrid_net_Seitenzahl', 'Karakas_net_yield_Portinari_net_Seitenzahl', 'Karakas_net_yield_TNG_net_Seitenzahl', 'Karakas_net_yield_West17_net_Seitenzahl', 'Karakas_net_yield_chieffi04_Seitenzahl', 'Karakas_net_yield_chieffi04_net_Seitenzahl', 'Nugrid_CL18_net_Seitenzahl', 'Nugrid_Frischknecht16_net_Seitenzahl', 'Nugrid_Nomoto2013_Seitenzahl', 'Nugrid_Nomoto2013_net_Seitenzahl', 'Nugrid_NuGrid_net_Seitenzahl', 'Nugrid_Portinari_net_Seitenzahl', 'Nugrid_TNG_net_Seitenzahl', 'Nugrid_West17_net_Seitenzahl', 'Nugrid_chieffi04_Seitenzahl', 'Nugrid_chieffi04_net_Seitenzahl', 'TNG_net_CL18_net_Seitenzahl', 'TNG_net_Frischknecht16_net_Seitenzahl', 'TNG_net_Nomoto2013_Seitenzahl', 'TNG_net_Nomoto2013_net_Seitenzahl', 'TNG_net_NuGrid_net_Seitenzahl', 'TNG_net_Portinari_net_Seitenz

In [7]:
MTf.train_models(path="data/big_MTf_model_comp", batch_size=512, device="cuda")

[rank0]:[W427 23:11:42.591615106 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


KeyboardInterrupt: 